## Evaluation using Mean Average Precision

In [ ]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

IMG_HEIGHT = 600
SIGN_SIZE = (32, 32)

# Function for reading the images
def readImages(rootpath, images_range, signs_range):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.
    Arguments: path to the traffic sign data, for example 'FullIJCNN2013'
    Returns:   list of images, list of corresponding labels'''
    images = {} # original image
    scales = {} # original scale
    for num in images_range:
        filename = rootpath + '/' + "{:05d}".format(num) + '.ppm'
        img = cv2.imread(filename, cv2.IMREAD_COLOR)
        scale = IMG_HEIGHT / float(img.shape[0])
        img_resized = cv2.resize(img, (int(img.shape[1]*scale),int(img.shape[0]*scale)))
        images.setdefault(filename,[]).append(img_resized)
        scales.setdefault(filename,[]).append(scale)

    files = [] # filenames
    signs = [] # traffic sign image
    bboxes = [] # corresponding box detection
    labels = [] # traffic sign type
    data = np.genfromtxt(rootpath + '/' + 'gt.txt', delimiter=';', dtype=str, usecols=range(0, 6))
    for elem in signs_range:
        filename = rootpath + '/' + data[elem][0]
        img = images.get(filename)[0]
        scale = scales.get(filename)[0]
        bbox = np.array([int(data[elem][1]), int(data[elem][2]), int(data[elem][3]), int(data[elem][4])]) * scale
        sign = img[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        sign_resized = cv2.resize(sign, SIGN_SIZE)
        files.append(filename)
        signs.append(sign_resized)
        bboxes.append(bbox)
        labels.append(data[elem][5])
    return images, files, signs, bboxes, labels

### Load BinaryClassifier

In [ ]:
from keras.models import load_model

#model_detector = load_model('binary_model.h5')
model_detector = load_model('binary_model_32.h5')

### Load SignsClassifier

In [ ]:
from keras.models import load_model

#model_classifier = load_model('resnet.h5')
model_classifier = load_model('classifier_32.h5')

### Load BoundingBoxes and data

In [ ]:
import pickle
    
with open('test_pred.pickle', 'rb') as f:
    test_pred = pickle.load(f)

In [ ]:
test_images, test_files, test_signs, test_bboxes, test_labels = readImages('FullIJCNN2013', range(600,900), range(852,1213))

### Bbox filters

In [ ]:
def filter_square(bbox):
    largo = abs(bbox[2]-bbox[0])
    ancho = abs(bbox[1]-bbox[3])
    proporcion = largo/ancho
    if(0.7 <= proporcion <= 1.3):
        return True
    else:
        return False
    
def filter_bigbox(bbox):
    largo = abs(bbox[2]-bbox[0])
    ancho = abs(bbox[1]-bbox[3])
    if(largo<150):
        return True 
    if(ancho<150):
        return True
    else:
        return False

In [ ]:
for filename in test_images:
    anns_ofs = open('mAP/ground-truth/' + filename[-9:-4:] + '.txt', 'w')
    if filename in test_files:
        for idx in [i for i, x in enumerate(test_files) if x == filename]:
            bbox = test_bboxes[idx]
            label = test_labels[idx]
            anns_ofs.write(str(label) + ' ' + str(bbox[0]) + ' ' + str(bbox[1]) + ' ' + str(bbox[2]) + ' ' + str(bbox[3]) + '\n')
    anns_ofs.close()

In [ ]:
'''
Note that in our code sign_class: represents the predicted traffic sign label, 
confidence: the confidence that the bbox has a traffic sign, and 
bbox[...]: are the bbox co-ordinates.
'''
from time import time
import matplotlib.pyplot as plt

start = time()

counter = 0

for filename in test_images:
    counter += 1
    prds_ofs = open('mAP/predicted/' + filename[-9:-4:] + '.txt', 'w')
    aux = test_images.get(filename)[0].copy()

    for bbox in test_pred.get(filename)[0]:
        roi = test_images.get(filename)[0][int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        roi_resized = cv2.resize(roi, SIGN_SIZE)
        roi_resized = roi_resized.astype('float32')
        roi_resized /= 255.0
        roi_resized = np.reshape(roi_resized, [1,SIGN_SIZE[0],SIGN_SIZE[1],3])
        # Run your own detector and save each traffic sign classification
        
        if(filter_square(bbox)):
            if(filter_bigbox(bbox)):
        
                sign_background = model_detector.predict_classes(roi_resized)
                confidence = model_detector.predict(roi_resized)
                #print(confidence)
        
        # ...
        # traffic_sign_class confidence bbox
                if sign_background == 1:
            
                    sign_class = model_classifier.predict(roi_resized)
            
                    prds_ofs.write(str(np.argmax(sign_class[0])) + ' ' + str(confidence[0][1]) + ' ' + str(bbox[0]) + ' ' + str(bbox[1]) + ' ' + str(bbox[2]) + ' ' + str(bbox[3]) + '\n')
                    cv2.rectangle(aux, (int(bbox[0]),int(bbox[1])), (int(bbox[2]),int(bbox[3])), (0,255,0), 3)
            
    # Para la cantidad de señales que haya en esta road image
    for idx in [i for i, x in enumerate(test_files) if x == filename]:
    # Coje las ground-truth bboxes
        bbox_truth = test_bboxes[idx]
        # Y dibujalas
        cv2.rectangle(aux, (int(bbox_truth[0]),int(bbox_truth[1])), (int(bbox_truth[2]),int(bbox_truth[3])), (255,0,0), 3)
    
    prds_ofs.close()
    print("Image number: {}".format(counter))
    plt.imshow(cv2.cvtColor(aux, cv2.COLOR_BGR2RGB))
    plt.show()
end = time()
print("Traffic sign detection took " + str(end - start) + " seconds")





### mAP Score

In [ ]:
# Follow previous mAP code in order to evaluate the performance of your neural net
%cd -q mAP
%run main.py -na -np
%cd -q